In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from os.path import join as p_join
import sys
from typing import List, Tuple, Dict, Set, Any, Optional, Callable
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import requests

try:
    from fabulous import color as fb_color
    color_print = lambda x, color='green': print(getattr(fb_color, color)(x)) if 'fb_color' in globals() else print(x)
except Exception as e:
    color_print = lambda x, color='green': print(x)

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [43]:
from src.parse_utils import get_events_list, get_fights_info
from src.processing import eventslist2df

In [4]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

---

In [42]:
fights_list, status_ok = get_events_list()
print(status_ok)
print(len(fights_list))
fights_list[:3]

True
613


[{'event_url': 'http://www.ufcstats.com/event-details/319c15b8aac5bfde',
  'event_name': 'UFC Fight Night: Blaydes vs. Aspinall',
  'date': 'July 23, 2022',
  'location': 'London, England, United Kingdom'},
 {'event_url': 'http://www.ufcstats.com/event-details/8fd76e1b49c00ae2',
  'event_name': 'UFC Fight Night: Ortega vs. Rodriguez',
  'date': 'July 16, 2022',
  'location': 'Elmont, New York, USA'},
 {'event_url': 'http://www.ufcstats.com/event-details/31da66df48c0c1a0',
  'event_name': 'UFC Fight Night: Dos Anjos vs. Fiziev',
  'date': 'July 09, 2022',
  'location': 'Las Vegas, Nevada, USA'}]

In [44]:
fights_df = eventslist2df(fights_list)
fights_df

,event_url,event_name,date,location
0,http://www.ufcstats.com/event-details/319c15b8...,UFC Fight Night: Blaydes vs. Aspinall,"July 23, 2022","London, England, United Kingdom"
1,http://www.ufcstats.com/event-details/8fd76e1b...,UFC Fight Night: Ortega vs. Rodriguez,"July 16, 2022","Elmont, New York, USA"
2,http://www.ufcstats.com/event-details/31da66df...,UFC Fight Night: Dos Anjos vs. Fiziev,"July 09, 2022","Las Vegas, Nevada, USA"
3,http://www.ufcstats.com/event-details/4a9e3056...,UFC 276: Adesanya vs. Cannonier,"July 02, 2022","Las Vegas, Nevada, USA"
4,http://www.ufcstats.com/event-details/eb42d4fe...,UFC Fight Night: Tsarukyan vs. Gamrot,"June 25, 2022","Las Vegas, Nevada, USA"
...,...,...,...,...
608,http://www.ufcstats.com/event-details/1c3f5e85...,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, USA"
609,http://www.ufcstats.com/event-details/dedc3bb4...,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, USA"
610,http://www.ufcstats.com/event-details/b60391da...,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, USA"
611,http://www.ufcstats.com/event-details/1a49e067...,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, USA"


In [241]:
fights_info_dict = get_fights_info(fights_urls=fights_df['event_url'].values.tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [06:09<00:00,  1.66it/s]
